# Text summarization (Sentences selection)
https://medium.com/datadriveninvestor/simple-text-summarizer-using-nlp-d8aaf5828e68

- This summarization technique consists in scoring every sentences (by summing words' scores) and selecting only the n-top scored sentences

- Score(word) = Term frequency in input text

- Benefits : 
    - Preserves the grammatical structure of the text (because the selected sentences are unchanged)
    - Easy to understand (low complexity)

- Drawbacks : 
    - Scoring method do not consider the sentence as a whole but as a list of independant words
    - Sentences's importance evaluation subject of discussion :
        - Is the sum of words' frequency in a sentence relevant for sentence importance ?
        - A long sentence is more likely to be selected because you sum up many terms' frequency, is it relevant ?

### 1 : Libraries import

In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from string import punctuation
from heapq import nlargest
from collections import defaultdict

In [2]:
stop_words = stopwords.words('french') + stopwords.words('english')

### 2 : Text to summarize

In [3]:
#text_2_summarize = 'The 2020 London Marathon will involve only elite athletes, with 45000 mass-event runners unable to take part because of coronavirus concerns. The much-anticipated contest between Kenya\'s Eliud Kipchoge and Ethiopian Kenenisa Bekele will take place on a bio-secure closed course. Reduced fields of 30-40 athletes will also compete for the elite women\'s and wheelchair titles on 4 October. The 2021 race, meanwhile, will be on 3 October rather than a date in April. That calendar shift from the traditional date is designed to maximise the chances of all runners being able to take part in next year\'s race.'
text_2_summarize = 'Le marathon de Londres 2020 impliquera uniquement des athlètes d\'élite, avec 45000 coureurs de \'\' mass-event \'\' incapables de participer en raison de problèmes de coronavirus. Le concours tant attendu entre Eliud Kipchoge du Kenya et l\'Ethiopien Kenenisa Bekele devait se dérouler sur un parcours fermé biosécurité. Des pelotons réduits de 30 à 40 athlètes concourront également pour les titres élite féminins et en fauteuil roulant. La course 2021, quant à elle, aura lieu le 3 octobre plutôt qu\'une date en avril. Ce changement de calendrier par rapport à la date traditionnelle est conçu pour maximiser les chances de tous les coureurs de participer à la course de l\'année prochaine.'

In [4]:
text_2_summarize

"Le marathon de Londres 2020 impliquera uniquement des athlètes d'élite, avec 45000 coureurs de '' mass-event '' incapables de participer en raison de problèmes de coronavirus. Le concours tant attendu entre Eliud Kipchoge du Kenya et l'Ethiopien Kenenisa Bekele devait se dérouler sur un parcours fermé biosécurité. Des pelotons réduits de 30 à 40 athlètes concourront également pour les titres élite féminins et en fauteuil roulant. La course 2021, quant à elle, aura lieu le 3 octobre plutôt qu'une date en avril. Ce changement de calendrier par rapport à la date traditionnelle est conçu pour maximiser les chances de tous les coureurs de participer à la course de l'année prochaine."

### 3 : Words' tokenization & processing (cleaning)

In [5]:
text_2_summarize = text_2_summarize.lower()
tokens = [t for t in text_2_summarize.split()]
print(tokens)

['le', 'marathon', 'de', 'londres', '2020', 'impliquera', 'uniquement', 'des', 'athlètes', "d'élite,", 'avec', '45000', 'coureurs', 'de', "''", 'mass-event', "''", 'incapables', 'de', 'participer', 'en', 'raison', 'de', 'problèmes', 'de', 'coronavirus.', 'le', 'concours', 'tant', 'attendu', 'entre', 'eliud', 'kipchoge', 'du', 'kenya', 'et', "l'ethiopien", 'kenenisa', 'bekele', 'devait', 'se', 'dérouler', 'sur', 'un', 'parcours', 'fermé', 'biosécurité.', 'des', 'pelotons', 'réduits', 'de', '30', 'à', '40', 'athlètes', 'concourront', 'également', 'pour', 'les', 'titres', 'élite', 'féminins', 'et', 'en', 'fauteuil', 'roulant.', 'la', 'course', '2021,', 'quant', 'à', 'elle,', 'aura', 'lieu', 'le', '3', 'octobre', 'plutôt', "qu'une", 'date', 'en', 'avril.', 'ce', 'changement', 'de', 'calendrier', 'par', 'rapport', 'à', 'la', 'date', 'traditionnelle', 'est', 'conçu', 'pour', 'maximiser', 'les', 'chances', 'de', 'tous', 'les', 'coureurs', 'de', 'participer', 'à', 'la', 'course', 'de', "l'anné

In [6]:
clean_token = tokens[:]
stopword = set(stop_words + list(punctuation))

for token in tokens:
    if token in stopword:
        clean_token.remove(token)

print(clean_token)

['marathon', 'londres', '2020', 'impliquera', 'uniquement', 'athlètes', "d'élite,", '45000', 'coureurs', "''", 'mass-event', "''", 'incapables', 'participer', 'raison', 'problèmes', 'coronavirus.', 'concours', 'tant', 'attendu', 'entre', 'eliud', 'kipchoge', 'kenya', "l'ethiopien", 'kenenisa', 'bekele', 'devait', 'dérouler', 'parcours', 'fermé', 'biosécurité.', 'pelotons', 'réduits', '30', '40', 'athlètes', 'concourront', 'également', 'titres', 'élite', 'féminins', 'fauteuil', 'roulant.', 'course', '2021,', 'quant', 'elle,', 'lieu', '3', 'octobre', 'plutôt', "qu'une", 'date', 'avril.', 'changement', 'calendrier', 'rapport', 'date', 'traditionnelle', 'conçu', 'maximiser', 'chances', 'tous', 'coureurs', 'participer', 'course', "l'année", 'prochaine.']


### 4 : Frequency distribution of the n most common words (= Bag of words)

In [7]:
freq = nltk.FreqDist(clean_token)
top_words = []
top_words = freq.most_common(20)
print(top_words)

[('athlètes', 2), ('coureurs', 2), ("''", 2), ('participer', 2), ('course', 2), ('date', 2), ('marathon', 1), ('londres', 1), ('2020', 1), ('impliquera', 1), ('uniquement', 1), ("d'élite,", 1), ('45000', 1), ('mass-event', 1), ('incapables', 1), ('raison', 1), ('problèmes', 1), ('coronavirus.', 1), ('concours', 1), ('tant', 1)]


### 5 : Sentences tokenization

In [8]:
sentences = sent_tokenize(text_2_summarize)
print(sentences)

["le marathon de londres 2020 impliquera uniquement des athlètes d'élite, avec 45000 coureurs de '' mass-event '' incapables de participer en raison de problèmes de coronavirus.", "le concours tant attendu entre eliud kipchoge du kenya et l'ethiopien kenenisa bekele devait se dérouler sur un parcours fermé biosécurité.", 'des pelotons réduits de 30 à 40 athlètes concourront également pour les titres élite féminins et en fauteuil roulant.', "la course 2021, quant à elle, aura lieu le 3 octobre plutôt qu'une date en avril.", "ce changement de calendrier par rapport à la date traditionnelle est conçu pour maximiser les chances de tous les coureurs de participer à la course de l'année prochaine."]


### 6 : Sentences ranking

In [9]:
ranking = defaultdict(int)

for i, sent in enumerate(sentences):
    for word in word_tokenize(sent.lower()):
        if word in freq:
            ranking[i] += freq[word]
    top_sentences_index = nlargest(10, ranking, ranking.get)
    
print(top_sentences_index)

[4, 0, 1, 2, 3]


### 7 : Resume printing by sentences selection

In [10]:
summary = [sentences[j] for j in sorted(top_sentences_index)]
print(summary)

["le marathon de londres 2020 impliquera uniquement des athlètes d'élite, avec 45000 coureurs de '' mass-event '' incapables de participer en raison de problèmes de coronavirus.", "le concours tant attendu entre eliud kipchoge du kenya et l'ethiopien kenenisa bekele devait se dérouler sur un parcours fermé biosécurité.", 'des pelotons réduits de 30 à 40 athlètes concourront également pour les titres élite féminins et en fauteuil roulant.', "la course 2021, quant à elle, aura lieu le 3 octobre plutôt qu'une date en avril.", "ce changement de calendrier par rapport à la date traditionnelle est conçu pour maximiser les chances de tous les coureurs de participer à la course de l'année prochaine."]


### 8 : Selection of important words in the selected sentences

In [26]:
summary_2 = []

for sent in summary:
    word_selected_in_sent = []
    
    for word in sent.split():
        if word.lower() not in stopword:
            word_selected_in_sent.append(word)

    summary_2.append(" ".join(word_selected_in_sent))

In [39]:
summary_2

["marathon londres 2020 impliquera uniquement athlètes d'élite, 45000 coureurs '' mass-event '' incapables participer raison problèmes coronavirus.",
 "concours tant attendu entre eliud kipchoge kenya l'ethiopien kenenisa bekele devait dérouler parcours fermé biosécurité.",
 'pelotons réduits 30 40 athlètes concourront également titres élite féminins fauteuil roulant.',
 "course 2021, quant elle, lieu 3 octobre plutôt qu'une date avril.",
 "changement calendrier rapport date traditionnelle conçu maximiser chances tous coureurs participer course l'année prochaine."]